In [34]:
# Team: Tethical Machine Learning
# Team members: Mellanie Martin, Wyatt Pigeon, Koby Grah

import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import TransformedTargetRegressor
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import MinMaxScaler

class SelectColumns( BaseEstimator, TransformerMixin ):
    def __init__( self, columns ):
        self.columns = columns

    def fit( self, xs, ys, **params ):
        return self
    
    def transform( self, xs ):
        return xs[ self.columns ]

if __name__ == "__main__":
    data = pd.read_csv("Data/small.csv")
    columns_to_combine = [col for col in data.columns if col.startswith('bg-')]
    drop = [col for col in data.columns if (col.startswith('activity-'))]

    combined_columns = []
    for i in range(0, len(columns_to_combine), 3):  # Every three columns
        group = columns_to_combine[i:i+3]  # Group 'em up
    
        if len(group) == 3:  # If actually got 3 columns
            new_column_name = f"{group[0]}-{group[2][-5:]}"  # Rename column group with first time and last time
            data[new_column_name] = data[group].bfill(axis=1).iloc[:, 0]  # Fill new column with whichever column actually has a value
            combined_columns.append(new_column_name)  # Add new column to combined list

    data = data.drop(columns=columns_to_combine)  # Drop original columns
    data = data.drop(columns=drop)  # If you need to drop anything for visibility
    data = data.fillna(0)
    desired_columns = ('bg-', 'insulin-', 'cals-', 'carbs-')
    
    x_names = [col for col in data.columns if col.startswith(desired_columns)]
    xs = data[x_names]
    ys = data['bg+1:00']
    
    bg = data.filter(like='bg-').columns
    cals = data.filter(like='cals').columns
    insulin = data.filter(like='insulin').columns
    carbs = data.filter(like = 'carbs').columns
    bg = bg.append(cals).append(insulin).append(carbs)
    bg = bg.tolist()
    regressor = TransformedTargetRegressor(
        GradientBoostingRegressor(criterion = 'squared_error'),
        func = np.sqrt,
        inverse_func = np.square
    )

    scaler = MinMaxScaler()
    steps = [
        #('scaler', scaler),
        ('column_select', SelectColumns([bg])),
        ('gradientBoost', regressor),
    ]
    pipe = Pipeline(steps)

    grid = { 
        'column_select__columns': [
            bg,
        ],
        'gradientBoost__loss': ["squared_error", "absolute_error"],
        'gradientBoost__min_samples_split': range(2, 3),
        'gradientBoost__max_depth': range(1, 10),
        'gradientBoost': [
        GradientBoostingRegressor(criterion = 'squared_error') # no transformation
        ]
    }

    search = GridSearchCV(pipe, grid, cv = 5, scoring = "r2", n_jobs = -1)
    search.fit(xs, ys)
    print("\nGradient Boosting:")
    print(f"R-squared: {search.best_score_}")
    print(f"Best params: {search.best_params_}")

/var/folders/rs/cnd6d11d1r11v_pbzqn883pc0000gn/T/ipykernel_84074/3186843802.py:24: DtypeWarning: Columns (436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("Data/small.csv")


KeyboardInterrupt: 

Notes:
1. Final predicted value is a float, so this is a regression problem
2. Potential models to look at; Kernel Ridge, AdaBoost, Gradient Boosting
3. Pipeline with minimum 3 stages is needed, and we need to use GridSearchCV